ilko's first Kaggle project - exploring instacart's public data

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
# Load input data files in separate DataFrame object
orders_prior = pd.read_csv('../input/order_products__prior.csv')
orders_train = pd.read_csv('../input/order_products__train.csv')
orders = pd.read_csv('../input/orders.csv')
products = pd.read_csv('../input/products.csv')
aisles = pd.read_csv('../input/aisles.csv')
departments = pd.read_csv('../input/departments.csv')

In [ ]:
# Retrieve a sample customer ids to make computation more manageable
sample_customers = orders['user_id'].unique()
rand_seed = np.random.RandomState(12345)

sample_cust = list(rand_seed.choice(sample_customers, 20000, replace=False))

orders = orders[orders['user_id'].isin(sample_cust)]

# Merge input files into the orders_prior DataFrame and drop unneeded columns
# We replace ids for aisles, products and departments with actual names
orders_prior_mrg = orders_prior.merge(products, on='product_id')
orders_prior_mrg = orders_prior_mrg.merge(aisles, on='aisle_id')
orders_prior_mrg = orders_prior_mrg.merge(departments, on='department_id')
orders_prior_mrg = orders_prior_mrg.merge(orders, how='right', on='order_id')

orders_prior_mrg.drop(['product_id', 'aisle_id', 'department_id']
                      , axis=1, inplace=True)
orders_prior_mrg.sort_values(by='order_id', inplace=True)

print(orders_prior_mrg.head())

In [ ]:
def proportion_of_type(data_df, target_df, column, val_df):
    categories = list(target_df[column].unique())
    
    for cat in categories:
        data_df[cat] = [(val_df[(val_df[column] == cat) & (val_df['user_id'] == row)].shape[0])
                      / (val_df[val_df['user_id'] == row].shape[0] * 1.0) for row in data_df['user_id']]
    return data_df

stuff = proportion_of_type(orders, departments, 'department', orders_prior_mrg)
print(stuff.head())